In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from docplex.mp.model import Model
from docplex.mp.utils import DOcplexException

In [ ]:
def shift_and_append(df, n):

    # Shift the DataFrame by n positions (downward)
    shifted_df = df.shift(n)
    
    # Collect the dropped values (the last n rows before the shift)
    dropped_values = df.iloc[:-n].reset_index(drop=True)
    
    # Replace the first n rows (now NaN) with the dropped values
    shifted_df.iloc[n:] = dropped_values.values
    
    return shifted_df

In [ ]:
def shift_and_concat(df, shifts):

    c = pd.DataFrame()
    noise = np.random.normal(0, 0.01, (df.shape[0]))
    
    for s in shifts:
        shifted_df = shift_and_append(df, s)
        shifted_df = shifted_df + noise
        c = pd.concat([c, shifted_df], axis=1)
        noise += np.random.normal(0, 0.01, (df.shape[0]))
    c.columns = [f'{df.name}_{abs(s)}h' for s in shifts]
    return c

In [ ]:
def optimize_energy(
    solar_profile, load_profile, price, emissions, batt_cap=6.5, weight_emissions=0.6, cost_margin = 0.1,
    nominal_pv_power: float = 1.0
):

    horizon = len(solar_profile)
    weight_price = 1 - weight_emissions
    sell_price = price * (1 - cost_margin)

    model = Model(name='self_sufficient_day')

    # Define the decision variables
    net_energy = model.continuous_var_list(horizon, name='net', lb=-100, ub=100)
    batt_ch = model.continuous_var_list(horizon, name='batt_ch', lb=0, ub=1)
    batt_dch = model.continuous_var_list(horizon, name='batt_dch', lb=-1, ub=0)
    soc = model.continuous_var_list(horizon, name='soc', lb=0, ub=1)

    # Energy balance
    for i in range(horizon):
        model.add_constraint(net_energy[i] == load_profile[i] + (batt_ch[i] + batt_dch[i]) * batt_cap - solar_profile[i] * nominal_pv_power)

    # Battery
    model.add_constraint(soc[0] == 0)  # Initial state of charge

    # Charge and discharge
    for i in range(1,horizon):
        model.add_constraint(soc[i] == (soc[i-1] + batt_ch[i] + batt_dch[i]))

    # Action constraints

    for i in range(horizon):
        model.add_constraint(batt_ch[i] <= 1 - soc[i])
        model.add_constraint(model.abs(batt_dch[i]) <= soc[i])

    # Define the objective function
    
    obj = []
    
    for i in range(horizon):

        obj.append(weight_price * price[i] * model.max(net_energy[i], 0)) 
        obj.append(weight_price * sell_price[i] * model.min(net_energy[i], 0))
        obj.append(weight_emissions * emissions[i] * model.max(net_energy[i], 0))

    model.minimize(model.sum(obj))

    # Solve the model

    model.solve(log_output=False)
    
    # Prepare solution
    
    sol = {
        'solar_generation': solar_profile * nominal_pv_power,
        'load': load_profile,
        'batt': np.zeros(horizon),
        'soc': np.zeros(horizon),
        'net': np.zeros(horizon),
        'action': np.zeros(horizon)
    }

    try:

        for v in model.iter_continuous_vars():
            if v.name is not None:
                index = int(v.name.split('_')[-1])
                if v.name.startswith('load'):
                    sol['load'][index] += v.solution_value
                if v.name.startswith('batt'):
                    sol['batt'][index] += v.solution_value * batt_cap
                    sol['action'][index] += v.solution_value
                if v.name.startswith('soc'):
                    sol['soc'][index] = v.solution_value
                if v.name.startswith('net'):
                    sol['net'][index] = v.solution_value

    except DOcplexException as e:

        # print(f"DOcplexException: {e.message}")

        return sol, False

    return sol, True


In [ ]:
def plot_energy_solution(sol):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
            
    ax1.plot(sol['load'], label='Load', marker='o')
    ax1.plot(sol['batt'], label='Battery energy', color='g', linestyle='--', marker='o')
    ax1.plot(sol['solar_generation'], label='Solar generation', marker='o')
    ax1.plot(sol['load'] + sol['batt'] - sol['solar_generation'], label='Total energy', color='m', linestyle='--', marker='o')

    ax2.plot(sol['soc'], label='State of charge', color='tab:red', marker='o')
    ax2.plot(sol['action'], label='Battery action', color='tab:purple', marker='o')

    ax1.set_xlabel('Time (hours)')
    ax2.set_xlabel('Time (hours)')
    ax1.set_ylabel('Energy (kWh)')
    ax2.set_ylabel('State of Charge')

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    ax1.grid()
    ax2.grid()

    ax1.set_xticks(range(0, 24))
    ax2.set_xticks(range(0, 24))

    plt.show()

In [ ]:
# Define solar energy generation values (in kWh) for each hour

# solar_generation = np.array([0, 0, 0, 0, 0, 0, 0, 0.5, 1.5, 3.0, 4.5, 5.0, 4.8, 4.0, 3.0, 1.5, 0.5, 0.1, 0, 0, 0, 0, 0, 0]) * 200

# solar_generation = np.array([  0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
#           0.     ,  13.82083,  58.45417,  91.12083, 103.21667,  90.8125 ,
#          96.45834, 239.975  , 162.25833, 415.49167, 294.30417, 153.09166,
#          17.27917,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ])

# solar_generation = np.array([0.        , 0.        , 0.        , 0.        , 0.        ,
#        0.        , 0.        , 0.01382083, 0.05845417, 0.09112083,
#        0.10321667, 0.0908125 , 0.09645834, 0.239975  , 0.16225833,
#        0.41549167, 0.29430417, 0.15309166, 0.01727917, 0.        ,
#        0.        , 0.        , 0.        , 0.        ])

solar_generation = np.array([
    0.        , 0.        , 0.        , 0.        , 0.        ,
    0.        , 0.01450417, 0.11152084, 0.27647083, 0.453025  ,
    0.6069625 , 0.71275   , 0.759825  , 0.7512625 , 0.689675  ,
    0.581275  , 0.422425  , 0.2426    , 0.08224167, 0.00623333,
    0.        , 0.        , 0.        , 0.        
])

load_profile = np.array([
    0.        , 0.        , 0.        , 0.        , 0.        ,
    0.        , 0.01450417, 0.11152084, 0.27647083, 0.453025  ,
    0.6069625 , 0.71275   , 0.759825  , 0.7512625 , 0.689675  ,
    0.581275  , 0.422425  , 0.2426    , 0.08224167, 0.00623333,
    0.        , 0.        , 0.        , 0.        
]) * 0.2

load_profile = np.array([
1.70941715404,
0.73603421895,
0.69320285551,
0.70370503567,
0.68983651105,
0.84927879977,
1.05179196487,
1.93690962635,
1.65993249703,
0.54666332016,
0.64441110234,
0.57404119332,
0.40796465840,
0.54403037251,
0.66221436413,
0.54546301269,
0.53741896702,
0.82227577292,
1.88018466920,
1.64880876688,
2.79839637820,
1.99949340286,
2.14809126110,
2.14809126110,
])
# solar_generation /= 1000

# Define an assignation of energy consumption by levels

price = np.array([
    0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21,
    0.21, 0.21, 0.21, 0.21, 0.21, 0.50, 0.50, 0.50, 0.50, 0.50, 0.21,
    0.21, 0.21
])

# price *= -10

emissions = np.array([
    0.13, 0.13, 0.13, 0.13, 0.13, 0.14, 0.16, 0.16, 0.16, 0.16, 0.16,
    0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.17, 0.17, 0.16, 0.16, 0.16,
    0.15, 0.13
]) * 20

batt_cap = 5
nominal_pv_power = 4
cost_margin = 0.1
weight_emissions = 0.5

sol, solved = optimize_energy(
    solar_profile=solar_generation, load_profile=load_profile, price=price, emissions=emissions,
    batt_cap=batt_cap, weight_emissions=weight_emissions, nominal_pv_power=nominal_pv_power, cost_margin=cost_margin)
print(f'Solved: {solved}')
plot_energy_solution(sol=sol)

In [ ]:
import os
import json
import shutil

import numpy as np
import pandas as pd
from tqdm import tqdm

from pathlib import Path
from citylearn.data import DataSet

def generate_optimal_data_from_ref(
    base_dataset: str = 'citylearn_challenge_2022_phase_all', dest_folder: str = 'data/opt_data/', building_no: int = 5,
    batt_cap: float = 6.5, weight_emissions: float = 0.1, cost_margin: float = 0.1
):

    # If the directory exists, delete it and create the subfolders
    
    shutil.rmtree(dest_folder, ignore_errors=True)
    
    # Creat destination folder exists including the subfolders

    Path(dest_folder).mkdir(parents=True, exist_ok=True)

    # Get reference schema

    schema = DataSet.get_schema(base_dataset)

    # Reduce the number of buildings to 1

    schema['buildings'] = {f'Building_{i + 1}': schema['buildings'][f'Building_{i + 1}'] for i in range(building_no)}

    # Extract the base weather, emissions and pricing data (doesn't change among buildings)

    base_weather = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['weather']))
    base_pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))
    base_emissions = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['carbon_intensity']))

    # Read the base schema and process the json

    # We need just to store the load and optimal actions for each building
    # 0 - Battery charge/discharge
    # 1 - SoC
    # 2 - Action

    building_data = np.array([[[0.] * 4 for _ in range(365 * 24)] for _ in range(building_no)])
    base_csvs = []

    for building_no, (building_name, info) in enumerate(schema['buildings'].items()):

        # Create custom building CSVs

        base_csv = pd.read_csv(os.path.join(schema['root_directory'], info['energy_simulation']))
        solar_generation = base_csv['solar_generation'].values / 1000
        load_profile = base_csv['non_shiftable_load'].values

        nominal_pv_power = info['pv']['attributes']['nominal_power']

        base_csvs.append(base_csv)

        # Pass one day at a time the solar generation from the building data

        with tqdm(total=365, desc=f'Processing Building {building_no + 1}') as pbar:

            for day in range(365):

                solved = False

                sol, solved = optimize_energy(
                    solar_profile=solar_generation[day * 24:(day + 1) * 24],
                    load_profile=load_profile[day * 24:(day + 1) * 24],
                    price=base_pricing['electricity_pricing'].values[day * 24:(day + 1) * 24],
                    emissions=base_emissions['carbon_intensity'].values[day * 24:(day + 1) * 24],
                    batt_cap=batt_cap,
                    weight_emissions=weight_emissions,
                    cost_margin=cost_margin,
                    nominal_pv_power=nominal_pv_power
                )

                building_data[building_no, day * 24:(day + 1) * 24, 0] += sol['batt']
                building_data[building_no, day * 24:(day + 1) * 24, 1] += sol['soc']
                building_data[building_no, day * 24:(day + 1) * 24, 2] += sol['action']

                pbar.set_postfix({'day': day, 'solved': "yes" if solved else "no"})

                pbar.update(1)

                if not solved:
                    print(f"Day {day} not solved for building {building_no + 1}")
                    return

    # Update the base csv with the new data

    for building_no, (building_name, _) in enumerate(schema['buildings'].items()):

        base_csvs[building_no].to_csv(os.path.join(dest_folder, f'{building_name}.csv'), index=False)

        # Save batt action and soc in a separate file to avoid issues loading CityLearn

        sol_csv = pd.DataFrame(columns=['batt', 'soc'], index=range(365 * 24))

        sol_csv['batt'] = building_data[building_no,:,0]
        sol_csv['soc'] = building_data[building_no,:,1]
        sol_csv['action'] = building_data[building_no,:,2]

        sol_csv.to_csv(os.path.join(dest_folder, f'sol_{building_name}.csv'), index=False)

        # Save noisy predictions (synthetic) for the building

        shifts = [-4, -6, -12, -24] # 4, 6, 12, 24 hours in the future

        shift_and_concat(
            df=base_csvs[building_no]['non_shiftable_load'],
            shifts=shifts
        ).clip(lower=0).to_csv(os.path.join(dest_folder, f'pred_{building_name}.csv'), index=False)

        # Update the schema with the new paths

        schema['buildings'][building_name]['energy_simulation'] = f'{building_name}.csv'

        # Update the schema to guarantee no degradation in the battery efficiency

        schema['buildings'][building_name]['electrical_storage']['attributes'] = {
            "capacity": batt_cap,
            "efficiency": 1.0,
            "capacity_loss_coefficient": 0.0,
            "loss_coefficient": 0.0,
            "nominal_power": batt_cap,
            "power_efficiency_curve": [[0, 1],[0.3, 1],[0.7, 1],[0.8, 1],[1, 1]],
            "capacity_power_curve": [[0.0, 1],[0.8, 1],[1.0, 1]],
        }

    # Write pricing and emissions data to the destination folder

    base_weather.to_csv(os.path.join(dest_folder, 'weather.csv'), index=False)
    base_pricing.to_csv(os.path.join(dest_folder, 'pricing.csv'), index=False)
    base_emissions.to_csv(os.path.join(dest_folder, 'carbon_intensity.csv'), index=False)

    # Save the new schema in the destination folder

    schema['root_directory'] = dest_folder

    with open(os.path.join(dest_folder, 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)


In [ ]:
import os
import json
import shutil

import numpy as np
import pandas as pd
from tqdm import tqdm

from pathlib import Path
from citylearn.data import DataSet

def generate_optimal_data_from_ref(
    base_dataset: str = 'citylearn_challenge_2022_phase_all', dest_folder: str = 'data/year_opt_data/', building_no: int = 5,
    batt_cap: float = 6.5, weight_emissions: float = 0.1, cost_margin: float = 0.1
):

    # If the directory exists, delete it and create the subfolders
    
    shutil.rmtree(dest_folder, ignore_errors=True)
    
    # Creat destination folder exists including the subfolders

    Path(dest_folder).mkdir(parents=True, exist_ok=True)

    # Get reference schema

    schema = DataSet.get_schema(base_dataset)

    # Reduce the number of buildings to 1

    schema['buildings'] = {f'Building_{i + 1}': schema['buildings'][f'Building_{i + 1}'] for i in range(building_no)}

    # Extract the base weather, emissions and pricing data (doesn't change among buildings)

    base_weather = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['weather']))
    base_pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))
    base_emissions = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['carbon_intensity']))

    # Read the base schema and process the json

    # We need just to store the load and optimal actions for each building
    # 0 - Battery charge/discharge
    # 1 - SoC
    # 2 - Action

    building_data = np.array([[[0.] * 4 for _ in range(365 * 24)] for _ in range(building_no)])
    base_csvs = []

    for building_no, (building_name, info) in enumerate(schema['buildings'].items()):

        # Create custom building CSVs

        base_csv = pd.read_csv(os.path.join(schema['root_directory'], info['energy_simulation']))
        solar_generation = base_csv['solar_generation'].values / 1000
        load_profile = base_csv['non_shiftable_load'].values

        nominal_pv_power = info['pv']['attributes']['nominal_power']

        base_csvs.append(base_csv)

        # Pass one day at a time the solar generation from the building data

        solved = False

        sol, solved = optimize_energy(
            solar_profile=solar_generation,
            load_profile=load_profile,
            price=base_pricing['electricity_pricing'].values,
            emissions=base_emissions['carbon_intensity'].values,
            batt_cap=batt_cap,
            weight_emissions=weight_emissions,
            cost_margin=cost_margin,
            nominal_pv_power=nominal_pv_power
        )

        if not solved:
            print(f"Not solved for building {building_no + 1}")
            return

    # Update the base csv with the new data

    for building_no, (building_name, _) in enumerate(schema['buildings'].items()):

        base_csvs[building_no].to_csv(os.path.join(dest_folder, f'{building_name}.csv'), index=False)

        # Save batt action and soc in a separate file to avoid issues loading CityLearn

        sol_csv = pd.DataFrame(columns=['batt', 'soc'], index=range(365 * 24))

        sol_csv['batt'] = sol['batt']
        sol_csv['soc'] = sol['soc']
        sol_csv['action'] = sol['action']

        sol_csv.to_csv(os.path.join(dest_folder, f'sol_{building_name}.csv'), index=False)

        # Save noisy predictions (synthetic) for the building

        shifts = [-4, -6, -12, -24] # 4, 6, 12, 24 hours in the future

        shift_and_concat(
            df=base_csvs[building_no]['non_shiftable_load'],
            shifts=shifts
        ).clip(lower=0).to_csv(os.path.join(dest_folder, f'pred_{building_name}.csv'), index=False)

        # Update the schema with the new paths

        schema['buildings'][building_name]['energy_simulation'] = f'{building_name}.csv'

        # Update the schema to guarantee no degradation in the battery efficiency

        schema['buildings'][building_name]['electrical_storage']['attributes'] = {
            "capacity": batt_cap,
            "efficiency": 1.0,
            "capacity_loss_coefficient": 0.0,
            "loss_coefficient": 0.0,
            "nominal_power": batt_cap,
            "power_efficiency_curve": [[0, 1],[0.3, 1],[0.7, 1],[0.8, 1],[1, 1]],
            "capacity_power_curve": [[0.0, 1],[0.8, 1],[1.0, 1]],
        }

    # Write pricing and emissions data to the destination folder

    base_weather.to_csv(os.path.join(dest_folder, 'weather.csv'), index=False)
    base_pricing.to_csv(os.path.join(dest_folder, 'pricing.csv'), index=False)
    base_emissions.to_csv(os.path.join(dest_folder, 'carbon_intensity.csv'), index=False)

    # Save the new schema in the destination folder

    schema['root_directory'] = dest_folder

    with open(os.path.join(dest_folder, 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)


In [ ]:
# Define an assignation of energy consumption by levels

generate_optimal_data_from_ref(
    batt_cap=4, building_no=2, weight_emissions=0.6, cost_margin=0.1
)
    

In [ ]:
# Check data generated picking random days from random buildings

day_index = 24 * 0
building_index = 0

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

building_name = f'Building_{building_index + 1}'
building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{building_name}.csv'))

solar_generation = building_csv['solar_generation'].values / 1000
load = building_csv['non_shiftable_load'].values
batt = sol_csv['batt'].values
soc = sol_csv['soc'].values
action = sol_csv['action'].values
battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

plot_energy_solution(sol={
    'batt': batt[day_index:day_index + 24],
    'soc': soc[day_index:day_index + 24],
    'action': action[day_index:day_index + 24],
    'load': load[day_index:day_index + 24],
    'solar_generation': solar_generation[day_index:day_index + 24]
})

In [ ]:
# Check net energy of whole year

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(len(schema['buildings'])):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{building_name}.csv'))

    solar_generation = building_csv['solar_generation'].values / 1000
    load = building_csv['non_shiftable_load'].values
    batt = sol_csv['batt'].values
    soc = sol_csv['soc'].values
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values
    
    cost_no_batt = (net_energy_no_batt.clip(min=0) * pricing + net_energy_no_batt.clip(max=0) * 0.9 * pricing).sum()
    cost = (net_energy.clip(min=0) * pricing + net_energy.clip(max=0) * 0.9 * pricing).sum()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).sum()
    emissions = (emissions * net_energy.clip(min=0)).sum()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt} - With batt: {emissions}')

In [ ]:
# Check net energy for a given period

day_index = 5832

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(len(schema['buildings'])):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{building_name}.csv'))

    solar_generation = building_csv['solar_generation'].values[day_index:day_index+24] / 1000
    load = building_csv['non_shiftable_load'].values[day_index:day_index+24]
    batt = sol_csv['batt'].values[day_index:day_index+24]
    soc = sol_csv['soc'].values[day_index:day_index+24]
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values[day_index:day_index+24]
    
    cost_no_batt = (net_energy_no_batt.clip(min=0) * pricing + net_energy_no_batt.clip(max=0) * 0.9 * pricing).sum()
    cost = (net_energy.clip(min=0) * pricing).sum() + (net_energy.clip(max=0) * 0.9 * pricing).sum()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values[day_index:day_index+24]

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).sum()
    emissions = (emissions * net_energy.clip(min=0)).sum()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt.mean()} - With batt: {emissions.mean()}')

In [ ]:
# Check net energy for a given period

day_index = 5832

with open('data/opt_data/schema.json', 'r') as f:
    schema = json.load(f)

for b_ix in range(len(schema['buildings'])):

    building_name = f'Building_{b_ix + 1}'
    building_csv = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings'][building_name]['energy_simulation']))
    sol_csv = pd.read_csv(os.path.join(schema['root_directory'], f'sol_{building_name}.csv'))

    solar_generation = building_csv['solar_generation'].values[day_index:day_index+24] / 1000
    load = building_csv['non_shiftable_load'].values[day_index:day_index+24]
    batt = sol_csv['batt'].values[day_index:day_index+24]
    soc = sol_csv['soc'].values[day_index:day_index+24]
    battery_capacity = schema['buildings'][building_name]['electrical_storage']['attributes']['capacity']

    # Compute net electricity

    net_energy_no_batt = load - solar_generation
    net_energy = load - solar_generation + batt * battery_capacity

    # Compute energy cost

    pricing = pd.read_csv(os.path.join(schema['root_directory'], schema['buildings']['Building_1']['pricing']))['electricity_pricing'].values[day_index:day_index+24]
    
    cost_no_batt = (net_energy_no_batt * pricing).sum()
    cost = (net_energy * pricing).sum()

    # Correct energy cost

    correct_cost_no_batt = (net_energy_no_batt.clip(min=0) * pricing + net_energy_no_batt.clip(max=0) * 0.9 * pricing).sum()
    correct_cost = (net_energy.clip(min=0) * pricing + net_energy.clip(max=0) * 0.9 * pricing).sum()

    # Compute carbon intensity

    emissions = pd.read_csv(os.path.join(schema['root_directory'], 'carbon_intensity.csv'))['carbon_intensity'].values[day_index:day_index+24]

    emissions_no_batt = (emissions * net_energy_no_batt.clip(min=0)).sum()
    emissions = (emissions * net_energy.clip(min=0)).sum()

    print(f'Building {b_ix + 1}')
    print(f'Avg. Hour Cost No batt: {cost_no_batt} - With batt: {cost}')
    print(f'Avg. Hour Correct Cost No batt: {correct_cost_no_batt} - With batt: {correct_cost}')
    print(f'Avg. Hour Emissions No batt: {emissions_no_batt} - With batt: {emissions}')

# Validation data from convex combination of houses

In [ ]:
def convex_combination_of_df(df1, df2, alpha):

    comb = df1 * alpha + df2 * (1 - alpha)
    comb = comb.astype(df1.dtypes)

    return comb

In [ ]:
import shutil

def create_eval_data_from_ref(ref_dir: str = 'data/opt_data/'):

    # Get schema file

    with open(os.path.join(ref_dir, 'schema.json'), 'r') as f:
        schema = json.load(f)

    # Read the original files in folder

    pred_files = []

    for csv_file in sorted(Path(ref_dir).glob("pred_*.csv")):
                    
        df = pd.read_csv(csv_file)
        pred_files.append(df)

    building_files = []

    for csv_file in sorted(Path(ref_dir).glob("Building_*.csv")):
                    
        df = pd.read_csv(csv_file)
        building_files.append(df)

    sol_files = []

    for csv_file in sorted(Path(ref_dir).glob("sol_*.csv")):
                    
        df = pd.read_csv(csv_file)
        sol_files.append(df)

    alpha = 0.4
    combinations = []

    for i in range(len(schema['buildings'])):

        # Pick two random buildings

        building_a, building_b = np.random.choice(len(schema['buildings']), 2, replace=False)

        combinations.append((building_a, building_b))

        # Convex combination of the predictions

        pred_1 = pred_files[building_a]
        pred_2 = pred_files[building_b]

        pred = convex_combination_of_df(pred_1, pred_2, alpha)

        # Convex combination of the solutions

        sol_1 = sol_files[building_a]
        sol_2 = sol_files[building_b]

        sol = convex_combination_of_df(sol_1, sol_2, alpha)

        # Convex combination of the buildings, but excluding the columns that should not be combined

        building_1 = building_files[building_a]
        building_2 = building_files[building_b]

        building = convex_combination_of_df(building_1, building_2, alpha)

        # Save the new files in the eval subfolder, first check that the folder exists

        Path(os.path.join(ref_dir, 'eval')).mkdir(parents=True, exist_ok=True)

        pred.to_csv(os.path.join(ref_dir, 'eval', f'pred_Building_{i + 1}.csv'), index=False)
        building.to_csv(os.path.join(ref_dir, 'eval', f'Building_{i + 1}.csv'), index=False)
        sol.to_csv(os.path.join(ref_dir, 'eval', f'sol_Building_{i + 1}.csv'), index=False)

    # Modify the schema to point to the new files

    schema['root_directory'] = os.path.join(ref_dir, 'eval')

    # Save schema

    with open(os.path.join(ref_dir, 'eval', 'schema.json'), 'w') as f:
        json.dump(schema, f, indent=4)

    # Save the rest of relevant files

    for file in ['weather.csv', 'pricing.csv', 'carbon_intensity.csv']:
        shutil.copyfile(os.path.join(ref_dir, file), os.path.join(ref_dir, 'eval', file))

    return combinations, pred_files, building_files, sol_files

combinations, pred_files, building_files, sol_files = create_eval_data_from_ref()

In [ ]:
# Compare in a plot the convex combination of the predictions with the prediction of the convex combination for a given day

day_index = 5832
building_index = 1

b1, b2 = combinations[building_index]

pred_1 = pred_files[b1]
pred_2 = pred_files[b2]

# Read the combination from csv

comb = pd.read_csv(f'data/opt_data/eval/pred_Building_{building_index + 1}.csv')

plt.plot(pred_1['non_shiftable_load_4h'][day_index:day_index + 24], label=f'Building {b1} - Load', marker='o')
plt.plot(pred_2['non_shiftable_load_4h'][day_index:day_index + 24], label=f'Building {b2} - Load', marker='o')
plt.plot(comb['non_shiftable_load_4h'][day_index:day_index + 24], label='Combination - Load', marker='o')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# Compare in a plot the convex combination of the buildings with the building of the convex combination for a given day

day_index = 57

b1, b2 = combinations[building_index]

building_1 = building_files[b1]
building_2 = building_files[b2]

# Read the combination from csv

comb = pd.read_csv(f'data/opt_data/eval/Building_{building_index + 1}.csv')

plt.plot(building_1['solar_generation'][day_index:day_index + 24], label=f'Building {b1} - Load', marker='o')
plt.plot(building_2['solar_generation'][day_index:day_index + 24], label=f'Building {b2} - Load', marker='o')
plt.plot(comb['solar_generation'][day_index:day_index + 24], label='Combination - Load', marker='o')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# Check data generated picking random days from random buildings

day_index = 4128
building_index = 0

comb = pd.read_csv(f'data/opt_data/eval/Building_{building_index + 1}.csv')
sol = pd.read_csv(f'data/opt_data/eval/sol_Building_{building_index + 1}.csv')

plot_energy_solution({
    'load': comb['non_shiftable_load'].values[day_index:day_index + 24],
    'batt': sol['batt'].values[day_index:day_index + 24],
    'soc': sol['soc'].values[day_index:day_index + 24],
    'action': sol['action'].values[day_index:day_index + 24],
    'solar_generation': comb['solar_generation'].values[day_index:day_index + 24]/1000
})

In [ ]:
pd.read_csv(f'data/opt_data/sol_{building_name}.csv').head(24)

In [ ]:
sol['soc'].values[day_index:day_index + 24]

In [ ]:
sol['batt'].values[day_index:day_index + 24]

In [ ]:
comb['non_shiftable_load'].values[day_index:day_index + 24]

In [ ]:
comb['solar_generation'].values[day_index:day_index + 24] / 1000